In [4]:
import random

import pandas as pd
import numpy as np
from scipy import stats

# from oauth2client.service_account import ServiceAccountCredentials 
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials 

In [5]:
CREDENTIALS_PATH_GOOGLE = '../google-credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET = '1b75J-QTGrujSgF9r0_JPOKkcXAwzFVwpETOAyVBw8ak'

In [6]:
# Load service account credentials.
__credentials = Credentials.from_service_account_file(CREDENTIALS_PATH_GOOGLE, scopes=SCOPES)

# Creates Google Sheets API (v4/latest) service.
service = build('sheets', 'v4', credentials=__credentials)

FileNotFoundError: [Errno 2] No such file or directory: '../google-credentials.json'

In [38]:
# Gets values from Ach! Musik: Notations sheet.
values = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET, range='Notations').execute()['values']
headers = values.pop(0)

In [39]:
# Format data as pd.DataFrame
data = pd.DataFrame(values, columns=headers)
data

,genre,sub_genre,artist,album,song,Qu,Gr,Vi,Ro,Sa,Gl,Rx,Cl,Lu
0,Rock,Alternative,4 Non Blondes,"Bigger, Better, Faster, More!",What's Up,"8,75","8,5",,"8,7",None,None,None,None,None
1,Pop,Synth,A-Ha,Hunting High and Low,Take on Me,8,,,9,None,None,None,None,None
2,Pop,Disco,Abba,Abba,"I do, I do, I do, I do, I do",8,"8,5",5,8,,,,,5
3,Pop,Disco,Abba,Abba,Mamma Mia,"8,5","8,75","6,5","8,3",,6,None,None,None
4,Pop,Disco,Abba,Abba,SOS,9,9,7,"8,4",,"6,25",None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051,Hip-Hop,Rap,"Dr. Dre, Snoop Dogg",2001,The Next Episode,,"8,2",None,None,None,None,None,None,None
2052,Hip-Hop,Rap,"Dr. Dre, Eminem",2001,Forgot About Dre,,7,None,None,None,None,None,None,None
2053,Hip-Hop,Rap,"Dr. Dre, Snoop Dogg",The Chronic,"Nuthin' But A ""G"" Thang",,"7,75",None,None,None,None,None,None,None
2054,Hip-Hop,Rap,2Pac,All Eyez On Me,Ambitionz Az A Ridah,,"7,75",None,None,None,None,None,None,None


In [40]:
# Saving as csv for later use
data.to_csv("../data/achmusik.csv", index=False, decimal=",")

In [7]:
data = pd.read_csv("../data/achmusik.csv")
data

,genre,sub_genre,artist,album,song,Qu,Gr,Vi,Ro,Sa,Gl,Rx,Cl,Lu
0,Rock,Alternative,4 Non Blondes,"Bigger, Better, Faster, More!",What's Up,"8,75","8,5",NaN,"8,7",NaN,NaN,NaN,NaN,NaN
1,Pop,Synth,A-Ha,Hunting High and Low,Take on Me,8,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN
2,Pop,Disco,Abba,Abba,"I do, I do, I do, I do, I do",8,"8,5",5,8,NaN,NaN,NaN,NaN,5
3,Pop,Disco,Abba,Abba,Mamma Mia,"8,5","8,75","6,5","8,3",NaN,6,NaN,NaN,NaN
4,Pop,Disco,Abba,Abba,SOS,9,9,7,"8,4",NaN,"6,25",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051,Hip-Hop,Rap,"Dr. Dre, Snoop Dogg",2001,The Next Episode,NaN,"8,2",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2052,Hip-Hop,Rap,"Dr. Dre, Eminem",2001,Forgot About Dre,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2053,Hip-Hop,Rap,"Dr. Dre, Snoop Dogg",The Chronic,"Nuthin' But A ""G"" Thang",NaN,"7,75",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2054,Hip-Hop,Rap,2Pac,All Eyez On Me,Ambitionz Az A Ridah,NaN,"7,75",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Getting the decimals right -- commas to points and no more Nones
data = data.set_index(["genre", "sub_genre", "artist", "album", "song"])
data.fillna(value="", inplace=True)

for i in range(data.columns.size):
    data[data.columns[i]] = data[data.columns[i]].str.replace(",", ".")
    data[data.columns[i]] = pd.to_numeric(data[data.columns[i]], errors='coerce')

In [17]:
kept_people = ["Qu", "Gr", "Vi", "Ro"]
default_grade = 5

# Keeping only present people at the hypothetical party!
data = data.filter(kept_people)

# Hard to do this shit inplace -- if no grades at all, give it a chance to play with default grade
data = data.dropna(how="all").append(data[data.isnull().all(axis=1)].fillna(default_grade))
data

Qu  \
genre sub_genre   artist            album           song                                
Pop   Disco       Abba              Abba            I do, I do, I do, I do, I do  8.0   
                                                    Mamma Mia                     8.5   
                                                    SOS                           9.0   
                                    Arrival         Knowing me, knowing you       7.5   
                                    Super Trouper   Lay all your love on me       8.0   
...                                                                               ...   
Metal Symphonique Within Temptation The Unforgiving Murder                        5.0   
                                                    Shot in the Dark              5.0   
                                                    Sinead                        5.0   
                                                    Stairway to the Skies         5.0   
                                                    Where is the Edge             5.0   

                                                                                    Gr  \
genre sub_genre   artist            album           song                                 
Pop   Disco       Abba              Abba            I do, I do, I do, I do, I do  8.50   
                                                    Mamma Mia                     8.75   
                                                    SOS                           9.00   
                                    Arrival         Knowing me, knowing you       8.50   
                                    Super Trouper   Lay all your love on me       8.50   
...                                                                                ...   
Metal Symphonique Within Temptation The Unforgiving Murder                        5.00   
                                                    Shot in the Dark              5.00   
                                                    Sinead                        5.00   
                                                    Stairway to the Skies         5.00   
                                                    Where is the Edge             5.00   

                                                                                   Vi  \
genre sub_genre   artist            album           song                                
Pop   Disco       Abba              Abba            I do, I do, I do, I do, I do  5.0   
                                                    Mamma Mia                     6.5   
                                                    SOS                           7.0   
                                    Arrival         Knowing me, knowing you       5.5   
                                    Super Trouper   Lay all your love on me       4.0   
...                                                                               ...   
Metal Symphonique Within Temptation The Unforgiving Murder                        5.0   
                                                    Shot in the Dark              5.0   
                                                    Sinead                        5.0   
                                                    Stairway to the Skies         5.0   
                                                    Where is the Edge             5.0   

                                                                                   Ro  
genre sub_genre   artist            album           song                               
Pop   Disco       Abba              Abba            I do, I do, I do, I do, I do  8.0  
                                                    Mamma Mia                     8.3  
                                                    SOS                           8.4  
                                    Arrival         Knowing me, knowing you       6.5  
                                    Super Trouper   Lay all your lov

## Score voting

In [28]:
COUNT_FACTOR = .1
COUNT_INHIB = len(kept_people) // 2
MIN_SCORE = 2
PLAYLIST_SIZE = 1600
ELIMINATING_GRADE = 4.6

# To avoid having to hard code the amount of columns for cases where the next cell is re-ran, we initialize columns
data["mean"] = 0
data["count"] = 0
data["score"] = 0
data["rank"] = 0

In [29]:
# Mean of all notes for each track
data["mean"] = data[data.columns[:-4]].mean(axis=1)

# Amount of notes for each track
data["count"] = data.count(axis=1) - 4

# Helping songs graded by more people in the group
data["score"] = data["mean"] + (COUNT_FACTOR * (data["count"] - COUNT_INHIB))

# Truncating to keep only the acceptable songs
data = data[data["score"] > MIN_SCORE]

data = data.sort_values("score", ascending=False)
data["rank"] = data["score"].rank(method="min")
data

Qu  \
genre   sub_genre   artist            album                        song                                  
Rock    NaN         Dire Straits      Dire Straits                 Sultans Of Swing              10.00   
                    The Beatles       Abbey Road                   Here Comes The Sun            10.00   
        Acoustic    Eric Clapton      Unplugged                    Layla                         10.00   
        Progressive Pink Floyd        Animals                      Pigs (Three Different Ones)    9.75   
        NaN         The Beatles       The Beatles                  While My Guitar Gently Weeps   9.75   
...                                                                                                ...   
Metal   Symphonique Within Temptation The Unforgiving              Shot in the Dark               5.00   
Pop     Indie       Charlotte Cardin  Main Girl                    Main Girl                      5.25   
Hip-Hop Punk        Stupeflip         The Hypnoflip Invasion       Dangereux !!                   5.25   
Reggae  Ska-Punk    The Skints        Swimming Lessons             Gets on Top                     NaN   
Pop     Dance       Madonna           Confessions on a Dance Floor Hung Up                        3.00   

                                                                                                   Gr  \
genre   sub_genre   artist            album                        song                                 
Rock    NaN         Dire Straits      Dire Straits                 Sultans Of Swing               NaN   
                    The Beatles       Abbey Road                   Here Comes The Sun            9.25   
        Acoustic    Eric Clapton      Unplugged                    Layla                          NaN   
        Progressive Pink Floyd        Animals                      Pigs (Three Different Ones)    NaN   
        NaN         The Beatles       The Beatles                  While My Guitar Gently Weeps  9.50   
...                                                                                               ...   
Metal   Symphonique Within Temptation The Unforgiving              Shot in the Dark              5.00   
Pop     Indie       Charlotte Cardin  Main Girl                    Main Girl                      NaN   
Hip-Hop Punk        Stupeflip         The Hypnoflip Invasion       Dangereux !!                   NaN   
Reggae  Ska-Punk    The Skints        Swimming Lessons             Gets on Top                    NaN   
Pop     Dance       Madonna           Confessions on a Dance Floor Hung Up                        NaN   

                                                                                                   Vi  \
genre   sub_genre   artist            album                        song                                 
Rock    NaN         Dire Straits      Dire Straits                 Sultans Of Swing               NaN   
                    The Beatles       Abbey Road                   Here Comes The Sun            9.50   
        Acoustic    Eric Clapton      Unplugged                    Layla                         9.75   
        Progressive Pink Floyd        Animals                      Pigs (Three Different Ones)    NaN   
        NaN         The Beatles       The Beatles                  While My Guitar Gently Weeps   NaN   
...                                                                                               ...   
Metal   Symphonique Within Temptation The Unforgiving              Shot in the Dark              5.00   
Pop     Indie       Charlotte Cardin  Main Girl                    Main Girl                      NaN   
Hip-Hop Punk        Stupeflip         The Hypnoflip Invasion       Dangereux !!                  5.00   
Reggae  Ska-Punk    The Skints        Swimming Lessons             Gets on Top                    NaN   
Pop     Dance       Madonna           Confessions on a Dance Floor Hung Up                     

In [56]:
# Removing tracks with at least one grade under the minimum required
data = data[data[data.columns[:-4]].min(axis=1) > ELIMINATING_GRADE]
data

Qu  \
genre  sub_genre   artist           album                 song                                  
Rock   NaN         Dire Straits     Dire Straits          Sultans Of Swing              10.00   
                   The Beatles      Abbey Road            Here Comes The Sun            10.00   
       Acoustic    Eric Clapton     Unplugged             Layla                         10.00   
       Progressive Pink Floyd       Animals               Pigs (Three Different Ones)    9.75   
       NaN         The Beatles      The Beatles           While My Guitar Gently Weeps   9.75   
...                                                                                       ...   
Reggae Ska-Punk    The Skints       Swimming Lessons      New Kind of Friend              NaN   
Rock   Hard Rock   Three Days Grace Three Days Grace      I Hate Everything About You    5.50   
       Alternative The Pretenders   Break Up the Concrete Break Up the Concrete           NaN   
Pop    Indie       Charlotte Cardin Main Girl             Main Girl                      5.25   
Reggae Ska-Punk    The Skints       Swimming Lessons      Gets on Top                     NaN   

                                                                                          Gr  \
genre  sub_genre   artist           album                 song                                 
Rock   NaN         Dire Straits     Dire Straits          Sultans Of Swing               NaN   
                   The Beatles      Abbey Road            Here Comes The Sun            9.25   
       Acoustic    Eric Clapton     Unplugged             Layla                          NaN   
       Progressive Pink Floyd       Animals               Pigs (Three Different Ones)    NaN   
       NaN         The Beatles      The Beatles           While My Guitar Gently Weeps  9.50   
...                                                                                      ...   
Reggae Ska-Punk    The Skints       Swimming Lessons      New Kind of Friend             NaN   
Rock   Hard Rock   Three Days Grace Three Days Grace      I Hate Everything About You    NaN   
       Alternative The Pretenders   Break Up the Concrete Break Up the Concrete          NaN   
Pop    Indie       Charlotte Cardin Main Girl             Main Girl                      NaN   
Reggae Ska-Punk    The Skints       Swimming Lessons      Gets on Top                    NaN   

                                                                                          Vi  \
genre  sub_genre   artist           album                 song                                 
Rock   NaN         Dire Straits     Dire Straits          Sultans Of Swing               NaN   
                   The Beatles      Abbey Road            Here Comes The Sun            9.50   
       Acoustic    Eric Clapton     Unplugged             Layla                         9.75   
       Progressive Pink Floyd       Animals               Pigs (Three Different Ones)    NaN   
       NaN         The Beatles      The Beatles           While My Guitar Gently Weeps   NaN   
...                                                                                      ...   
Reggae Ska-Punk    The Skints       Swimming Lessons      New Kind of Friend             NaN   
Rock   Hard Rock   Three Days Grace Three Days Grace      I Hate Everything About You    NaN   
       Alternative The Pretenders   Break Up the Concrete Break Up the Concrete         5.50   
Pop    Indie       Charlotte Cardin Main Girl             Main Girl                      NaN   
Reggae Ska-Punk    The Skints       Swimming Lessons      Gets on Top                    NaN   

                                                                                         Ro  \
genre  sub_genre   artist           album                 song                                
Rock   NaN         Dire Straits     Dire Straits          Sultans Of Swing              NaN   
                   The Beatles      Abbey Road

In [224]:
if PLAYLIST_SIZE < 1:
    playlist = data.sample(frac=PLAYLIST_SIZE, weights="rank")
else:
    playlist = data.sample(n=PLAYLIST_SIZE, weights="rank")

playlist

Qu  \
genre   sub_genre   artist             album                   song                      
Metal   Power       Powerwolf          The Sacrament of Sin    Stossgebet         9.00   
        Groove      Pantera            Cowboys from Hell       Cowboys from Hell   NaN   
        Progressive Mastodon           Crack the Skye          The Last Baron     9.00   
Jazz    OST         Kristofer Maddigan Cuphead                 Die House          8.75   
Metal   Folk        Týr                Valkyrja                Mare of My Night    NaN   
...                                                                                ...   
        Industrial  Rammstein          Rosenrot                Ein Lied           6.75   
Chanson Rock        Renaud             Mistral Gagnant         Tu vas au bal ?    6.50   
Dub     NaN         Stand High Patrol  Summer on Mars          Fragile            4.25   
Metal   Blues       Black Sabbath      Black Sabbath           Warning            7.00   
Hip-Hop NaN         Caballero          Laisse Nous Faire Vol.1 Viens Voir         6.00   

                                                                                   Gr  \
genre   sub_genre   artist             album                   song                     
Metal   Power       Powerwolf          The Sacrament of Sin    Stossgebet         8.7   
        Groove      Pantera            Cowboys from Hell       Cowboys from Hell  9.0   
        Progressive Mastodon           Crack the Skye          The Last Baron     8.0   
Jazz    OST         Kristofer Maddigan Cuphead                 Die House          NaN   
Metal   Folk        Týr                Valkyrja                Mare of My Night   NaN   
...                                                                               ...   
        Industrial  Rammstein          Rosenrot                Ein Lied           NaN   
Chanson Rock        Renaud             Mistral Gagnant         Tu vas au bal ?    NaN   
Dub     NaN         Stand High Patrol  Summer on Mars          Fragile            NaN   
Metal   Blues       Black Sabbath      Black Sabbath           Warning            NaN   
Hip-Hop NaN         Caballero          Laisse Nous Faire Vol.1 Viens Voir         NaN   

                                                                                   Vi  \
genre   sub_genre   artist             album                   song                     
Metal   Power       Powerwolf          The Sacrament of Sin    Stossgebet         NaN   
        Groove      Pantera            Cowboys from Hell       Cowboys from Hell  NaN   
        Progressive Mastodon           Crack the Skye          The Last Baron     NaN   
Jazz    OST         Kristofer Maddigan Cuphead                 Die House          9.0   
Metal   Folk        Týr                Valkyrja                Mare of My Night   NaN   
...                                                                               ...   
        Industrial  Rammstein          Rosenrot                Ein Lied           NaN   
Chanson Rock        Renaud             Mistral Gagnant         Tu vas au bal ?    NaN   
Dub     NaN         Stand High Patrol  Summer on Mars          Fragile            NaN   
Metal   Blues       Black Sabbath      Black Sabbath           Warning            NaN   
Hip-Hop NaN         Caballero          Laisse Nous Faire Vol.1 Viens Voir         6.0   

                                                                                   Ro  \
genre   sub_genre   artist             album                   song                     
Metal   Power       Powerwolf          The Sacrament of Sin    Stossgebet         9.2   
        Groove      Pantera            Cowboys from Hell       Cowboys from Hell  NaN   
        Progressive Mastodon           Crack the Skye          The Last Baron     NaN   
Jazz    OST         Kristofer Maddigan Cuphead                 Die House          9.0   
Metal   Folk        Týr                Valkyrja                

## genre matrix and distances

In [31]:
genres_list = data.reset_index()[["genre", "sub_genre"]]
genres = genres_list["genre"].unique()
sub_genres = genres_list["sub_genre"].unique()
genres

array(['Rock', 'Pop', 'Disco', 'Metal', 'Hip-Hop', 'Jazz', 'Blues',
       'Chanson', 'R&B', 'Country', 'Funk', 'Folk', 'Classique', 'OST',
       'Dance', 'Reggae', 'Electro', 'Musical', 'Acapella'], dtype=object)

In [36]:
test = pd.DataFrame(columns=sub_genres)
test = test.reindex(test.index.tolist() + list(sub_genres))
test

,Alternative,Synth,Disco,NaN,Hard Rock,Heavy Metal,Glam,Grunge,Rap,Indie,...,Alternatif,Psychedelia,New Wave,Surf,Barbershop,Bluegrass,Reggae,Ska-Punk,Parodique,Heavy
Alternative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Synth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Disco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hard Rock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bluegrass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Reggae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ska-Punk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Parodique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Playground

In [42]:
data = data.reset_index()

In [150]:
COL = ["Qu", "Gr", "Vi"]
BY = "artist"
AMNT = 10

best = data[[BY, *COL]].dropna(how="any").groupby(BY).filter(lambda x: len(x) >= AMNT).groupby(BY).mean()[COL]
best[COL].mean(axis=1).sort_values(ascending=False).head(10)

artist
The Beatles         8.296474
The Doors           8.238636
Pink Floyd          8.069444
System Of A Down    7.986667
Ghost               7.800000
Daft Punk           7.732143
Gojira              7.666667
Gorillaz            7.601852
Muse                7.568182
Alice in Chains     7.282051
dtype: float64